In [4]:
from ultralytics import YOLO
import cv2

# Load model YOLOv8 pre-trained
model = YOLO("yolov8n.pt")  # Hoặc yolov8s.pt, yolov8m.pt, tùy yêu cầu

# Đọc ảnh
image_path = "cat.jpg"
image = cv2.imread(image_path)

# Dự đoán
results = model(image)

# Vẽ bounding box lên ảnh
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Lấy tọa độ box
        conf = box.conf[0].item()  # Độ chính xác
        label = result.names[int(box.cls[0])]  # Tên vật thể
        
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{label} {conf:.2f}", (x1, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Hiển thị ảnh
cv2.imshow("YOLO Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 640x640 1 clock, 135.4ms
Speed: 5.6ms preprocess, 135.4ms inference, 8.2ms postprocess per image at shape (1, 3, 640, 640)


In [60]:
import cv2
import numpy as np

def count_objects_edge_based(image_path):
    # Đọc ảnh
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Làm mịn ảnh để giảm nhiễu
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Tính trung vị của ảnh
    median_value = np.median(blurred)
    low_threshold = int(0.33 * median_value)
    high_threshold = int(1.33 * median_value)
    edges = cv2.Canny(blurred, low_threshold, high_threshold)
    
    # Tạo kernel (structuring element)
    kernel = np.ones((1, 1), np.uint8)
    # Áp dụng phép mở (Opening) để loại bỏ chi tiết nhỏ
    processed = cv2.morphologyEx(edges, cv2.MORPH_OPEN, kernel)

    kernel = np.ones((3, 3), np.uint8)
    # Thực hiện phép dilation
    dilated = cv2.dilate(processed, kernel, iterations=1)

    # Tìm contours từ biên cạnh
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Vẽ contours lên ảnh gốc
    result = image.copy()
    cv2.drawContours(result, contours, -1, (0, 255, 0), 2)
    
    # Đếm số lượng đối tượng
    num_objects = len(contours)
    print(f"Số lượng đối tượng đếm được: {num_objects}")
    
    # Hiển thị kết quả
    #cv2.imshow("Edge-based Detection", dilated)
    cv2.imshow("Edge-based Detection", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return num_objects

# Gọi hàm với đường dẫn ảnh
image_path = "cat.jpg"  # Thay bằng đường dẫn ảnh của bạn
count_objects_edge_based(image_path)


Số lượng đối tượng đếm được: 59


59

In [ ]:
import cv2
import numpy as np

# Đọc ảnh xám
img = cv2.imread("cat.jpg", cv2.IMREAD_GRAYSCALE)
#img = cv2.imread("cat.jpg")

# Tạo kernel (cấu trúc hình dạng để erode)
kernel = np.ones((3, 3), np.uint8)

# Thực hiện phép erosion
eroded = cv2.erode(img, kernel, iterations=1)

edges = img - eroded

# Hiển thị kết quả
cv2.imshow("Original", img)
cv2.imshow("Eroded", edges)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [69]:
import cv2
import numpy as np

# Đọc ảnh xám
img = cv2.imread("rabbit.jpeg", cv2.IMREAD_GRAYSCALE)

#dilation
kernel = np.ones((5, 5), np.uint8)
dilated = cv2.dilate(img, kernel, iterations=1)

#erode
kernel = np.ones((3, 3), np.uint8)
eroded = cv2.erode(dilated, kernel, iterations=1)

# Xóa cạnh bằng cách trừ ảnh gốc
#edges_removed = dilated - img

median_value = np.median(dilated)
low_threshold = int(0.33 * median_value)
high_threshold = int(1.33 * median_value)
edges = cv2.Canny(eroded, low_threshold, high_threshold)

contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Vẽ contours lên ảnh gốc
result = img.copy()
cv2.drawContours(result, contours, -1, (0, 255, 0), 2)

# Đếm số lượng đối tượng
num_objects = len(contours)
print(f"Số lượng đối tượng đếm được: {num_objects}")

# Hiển thị kết quả
cv2.imshow("hmmmm", eroded)
cv2.imshow("Dilated", dilated)
cv2.imshow("Edge-based Detection", result)
cv2.waitKey(0)
cv2.destroyAllWindows()


Số lượng đối tượng đếm được: 125


In [62]:
import cv2
import numpy as np

# 1. Đọc ảnh
img = cv2.imread("cat.jpg")

# 2. Tạo mặt nạ xác định vùng màu đen và màu gần đen
threshold = 50  # Ngưỡng để xác định màu gần đen
mask_black = np.all(img < threshold, axis=-1)

# 3. Chuyển tất cả màu không phải đen (hoặc gần đen) thành xanh lá
img[~mask_black] = [0, 255, 0]

# 4. Chuyển ảnh sang xám
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 5. Xác định vùng không phải màu xanh lá
green_gray_value = gray[~mask_black][0]  # Giá trị xám của màu xanh lá
mask_not_green = gray != green_gray_value

# 6. Chuyển các vùng không phải xanh lá thành trắng
gray[mask_not_green] = 255


median_value = np.median(dilated)
low_threshold = int(0.33 * median_value)
high_threshold = int(1.33 * median_value)
edges = cv2.Canny(gray, low_threshold, high_threshold)

# Hiển thị kết quả
cv2.imshow("Green Masked", edges)
cv2.imshow("Final Gray", gray)
cv2.waitKey(0)
cv2.destroyAllWindows()
